In [ ]:
# These imports are all the necessary imports that are done to make the program work.
from pyswip.prolog import Prolog
from pyswip.easy import *
#importing widgets to make the game have a gui.
import ipywidgets as widgets 
from IPython.display import display
from functools import partial
from ipywidgets import widgets, HBox, VBox, Layout
import random


#Initializing the minmiax
known = Functor("known",2)
prolog = Prolog() # this is the global for this code. Whenever I use "prolog", the interpreter 
                  # will call the Prolog() function, which will help us with our "computer side" of the game
prolog.consult("kb.pl") # This opens up the KB, which is a prolog wile with .pl as its extension.
retractall = Functor("retractall")


class TicTac:

    #This function initializes the game, particularly the size of the board. Takes in 3 arguements, 
    #self, n (which is the number of rows or collumns our board will have), and the sign that the player (user) is going
    #to play with
    def __init__(self, n, player='x'):
        self.n = n
        self.opponent = "x" if player == "o" else "o"
        self.color = player
        # x_next is a variable that decides if the next move is going to be for the player whose mark is x.
        # Randomizing it adds a bit fun as at the begining of the game, no matter what sign the human player chooses
        #either the human or the computer get to go first.
        if random.randint(0,1) <= 0.5:
            self.x_next = True
        else:
            self.x_next = False
        self.buttons = []
        self.board = [['.'] * n for i in range(n)] #fills up our 2D List with temporary values of '.'
        self.draw_buttons()
        self.i = n**2 #this is the number of boxes we create for the board.
        self.state = "game"
       
    #Prolog is set up in a way that it sees itself as a player and us as a opponent
        call(retractall(known))
        prolog.asserta("known(opponent, " + str(self.color) + ")") #This tells prolog what sign we are 
        prolog.asserta("known(player, " + str(self.opponent) + ")") #This tells prolog what sign it is
        self.next_turn()
        
  #This function does the work of displaying the widgets on the screen as part of the GUI. 
    def draw_buttons(self):
        
        for i in range(self.n**2): #creates the buttons to fill the board. n is the value input of the user.
                                   #so if user puts 4 as the value, n**2 (4 by 4) buttons will be created. 
            button = widgets.Button(description='', #code for how the button would look like and how it 
                                    disabled=False, #would work
                                    button_style='success',
                                    tooltip='Click',
                                    border='solid',
                                    icon='',
                                    layout=Layout(height='60px', width='60px'))
            self.buttons.append(button) #adds the buttons to the current empty buttons list
            button.on_click(partial(self.button_clicked, i)) #calls the button_clicked function with
                                                             #the index of the button passed as an arguement
        
        #The line below creates a textbox at the top of the game window, which tells us whose turn it is and at the end
        #if we lost against the computer or not.
        self.text = widgets.Text(value = 'Its your turn! Please select a box to mark your sign!', 
                                 layout=Layout(width='400px', height='60px'))
        
        tictactoe_board = VBox([HBox([self.buttons[i] for i in range(j*self.n, j*self.n+self.n)]) for j in range(self.n)])
        display(VBox([self.text, tictactoe_board])) #adds the buttons to the board. HBox and Vbox are special functions
                                                    #that allow flexible css styling capabilities.
        
    # The following function is what takes place when the user (a human player) clicks a button. 
    # As soon as the button is clicked, i (the variable counting how many boxes are left unmarked) decreases
    # by 1. 
    def button_clicked(self, index, button):
        #since our game board is essentially a 2D List, we need some sytem to idenify which position on the 2D List
        #has been filled. For that purposes, we take the index of the button and the number of rows/collums
        #and then y gives us either 0 or 1 (mod function) while x gives us a integer value.
        y = index%self.n 
        x = int(index/self.n)

        button.description = "x" if self.x_next else "o" #describing the button to be x or o.
        button.disabled = True #disables the button that has been clicked so that it cannot be used for this round.
        self.board[x][y] = "x" if self.x_next else "o" #fills the 2D List postion [x][y] with the choosen sign
        self.i -= 1 #reduces the number of buttons by one so that we know when all the boxes are marked.
        self.x_next = not self.x_next #changes the value from True to False so that the opponent can have their turn.
        
        win = [x for x in prolog.query(
            "winBoard(" + str(self.board).replace('\'','') + "," + self.color + ").", 
            maxresult=1
        )] #queries prolog to see if the current move made the state of the game change to win or not.
        
        if win==True:
            self.state = "win" #if prolog answers True to the above query, the status of the game is changed to win
                               #and then when the next_turn function is called, it would end the game by checking
                               #the current state and executing the one of the arguements.
        
        self.next_turn()

    #This function, much like the other function, decides where the computer (a.k.a. Prolog) will place it's mark.
    #It contains a depth heuristic which helps the minimax algorithm work. 
    def prolog_turn(self):
        for button in self.buttons:
            # This freezes board until the next turn starts (a.k.a. the minimax algorithm spits out something that)
            #we could work with.
            button.disabled = True
            
        # This calcualtes the depth of the minimax algorithm so that prolog can make the next move.
        # The depth for this minimax algorithm is terminated at 90000 because when larger boards
        # are drawn (eg. 4*4), the game slows down incredibly as it takes time for prolog to search 
        # through all those states and choose the best one. Faster computer turn times can be 
        # achieved by either decreasing the recursion depth or applying some other heuristic.
        depth = 1
        i = self.i
        work = i
        while work < 90000 and i > 0:
            work = work * i
            depth += 1
            i -= 1
        # The following lines are where this python program queries the KB to check what the next move for the computer
        # should be. 
        result = [x for x in prolog.query(
            "nextmove([" + str(self.board).replace('\'','') + "," + self.opponent + 
            " , game], [B, P, S], Z, " + str(depth) + ").", 
            maxresult=1 # Taking only one result as the computer can only make one move at a time.
        )][0]
        
        # Once the results get back, it is time to analyse it and mark the approprate position on the board with the 
        # computers chosen sign.
        self.state = result['S']
        self.board = [[str(item) for item in row] for row in result['B']]
        self.draw_board() # The 2D List is converted to a ipywidget board so that the user can see where the computer
                          # marked its move.
        self.i -= 1 # Decreasing the number of empty boxes by one as computer occupied one of them.
        self.x_next = not self.x_next 
        self.next_turn() #calling the next turn function so that user can take the turn.
        
    # This function checks the current state of the game and mainly deals with processing text outputs based on the 
    # results. It also controls the disabling of the board while the computer is making its turn and disabling
    # the already marked boxes. Finally, it also controls turn switching (user to computer and vice versa)
    def next_turn(self):
        # There are two ways the game could get draw, either all the boxes are marked up or the state of the game
        # is "draw". In any of those cases, the board is disabled and the game ends.
        if self.state == "draw" or self.i == 0:
            for button in self.buttons:
                button.disabled = True  
            self.text.value = 'It\'s a draw between you and the computer! Well played!'
            
        # If the state of the game is win, then first the rest of the buttons (if any) are disabled so that 
        # prolog/make a move.
        elif self.state == "win":
            for button in self.buttons:
                button.disabled = True
                
            # If x_next is true and player's sign is x or if x_next is False and player's sign is o
            # then prolog wins. Otherwise, the player wins.
            if (self.x_next and self.color == "x") or (not self.x_next and self.color == "o"):
                self.text.value = "Uh oh. You lost. Try again by rerunning the program"
            else:
                self.text.value = "Wow! You beat the AI! Incredible! Congratulations!"
                
        # If state is neither draw nor win, it stays as "game" and the game goes on.
        else:
            if (self.x_next and self.color == "x") or (not self.x_next and self.color == "o"):
                self.text.value = 'It\'s your turn! Please select a box to mark your sign!'
            else:
                self.text.value = 'It\'s computer\'s turn, please wait while computer makes a move'
                self.prolog_turn() # This is where the function for prologs turn is called. 
                                                 
    # As I mentioned earlier, the game is in the form of a 2D List and in order to visualize it, we need to convert
    # it into a visually pleasing and realistic looking board of boxes that the user can click. This is the
    # function that does that by taking the i and j values from the 2D List.
    def draw_board(self):
        for i in range(self.n):
            for j in range(self.n):
                if self.board[i][j] != ".": # If a certain cell is not empty (a.k.a. '.'), it is obvious that 
                                            # it will be marked with either x or o. So the box corresponding
                                            # to that cell would have its sign and would also be disabled.
                    self.buttons[i*self.n + j].description = self.board[i][j]
                    self.buttons[i*self.n + j].disabled = True
                else:
                    self.buttons[i*self.n + j].disabled = False # If any cell of the 2D List is empty, then the 
                                                                # corresponding box would be clickable to mark it.

# Finally, you need to say how big of a board you want, and what mark (x or o) you want to start with.
# For example, if you want a 4 by 4 board (16 boxes), and you want to start with o, then you need to write
# 4 and 'o' and it should look like: start = TicTac(4, 'o'). Also, a point to note is that if you want to 
# always get the first move, you would need to delete the if statement from the __init__ function and either set
# x_next = True and start with x or set x_next = False and start with o.

start = TicTac(5, 'x')